In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.spatial.transform import Rotation as R
import sympy
from tqdm.notebook import trange

# リー微分で簡単な勾配降下法を試す

In [2]:
roll = 30 #deg
pitch = 20 #deg
yaw = 10 #deg

x = sympy.Symbol('x')
y = sympy.Symbol('y')
z = sympy.Symbol('z')
a = sympy.Symbol('a')
b = sympy.Symbol('b')
c = sympy.Symbol('c')
k = sympy.Symbol('k')
l = sympy.Symbol('l')
m = sympy.Symbol('m')


phi = sympy.Matrix([a, b, c])
p = sympy.Matrix([x, y, z])
p2 = p.subs(x, 4).subs(y, 5).subs(z, 6)
phi1 = phi.subs(a, np.deg2rad(roll)).subs(b, np.deg2rad(pitch)).subs(c, np.deg2rad(yaw))

# SO(3)回転行列
Rx = sympy.Matrix(
    [[1, 0, 0],
     [0, sympy.cos(a), -sympy.sin(a)], 
     [0, sympy.sin(a), sympy.cos(a)],
    ]
)
Ry = sympy.Matrix(
    [[sympy.cos(b), 0, sympy.sin(b)],
     [0, 1, 0],
     [-sympy.sin(b), 0, sympy.cos(b)]
    ]
)
Rz = sympy.Matrix(
    [[sympy.cos(c), -sympy.sin(c), 0],
     [sympy.sin(c), sympy.cos(c), 0],
     [0, 0, 1]
    ]
)
# print("目標回転行列:")
rot = Rz*Ry*Rx
rot1 = sympy.trigsimp(rot.subs(a, phi1[0]).subs(b, phi1[1]).subs(c, phi1[2]))
# display(rot1)

A = sympy.Matrix([[0, -m, l],[m, 0, -k], [-l, k, 0]])
p2_wedge = sympy.matrix2numpy(A.subs(k, p2[0]).subs(l, p2[1]).subs(m, p2[2]))
p1 = sympy.matrix2numpy((rot*p2).subs(a, phi1[0]).subs(b, phi1[1]).subs(c, phi1[2]))
p2 = sympy.matrix2numpy(p2)

$\def\bm{\boldsymbol}$
$L=||p_1 - R(\bm{\theta})p_2||^2$の勾配を使って適当な$\bm{\theta}$から$L$を最小化する$\bm{\theta}$を獲得してみる

勾配は
$$
\nabla L = p_1^T R(\bm{\theta})[p_2]_{\times}
$$

In [14]:
alpha = 0.01
omega = np.ones(3)
print("初期回転ベクトル:{}".format(omega))
# 勾配降下法
for i in trange(1000):
    rot = R.from_rotvec(omega)
    d_theta = -alpha * p1.T @ rot.as_matrix().reshape(3,3) @ p2_wedge
    tmp = R.from_rotvec(d_theta)
    tmp = tmp.as_matrix() @ rot.as_matrix()
    tmp = R.from_matrix(tmp)
    omega = tmp.as_rotvec()

初期回転ベクトル:[1. 1. 1.]


  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
print("収束後回転ベクトル:{}".format(omega[0]))

収束後回転ベクトル:[0.4303644  0.3436757  0.00758861]


In [16]:
rot_disented = R.from_rotvec(omega[0]).as_matrix()
rot_disented @ p2

array([[6.06294170799353],
       [2.41970657778377],
       [5.86393706675834]], dtype=object)

In [17]:
p1

array([[6.06294170799353],
       [2.41970657778377],
       [5.86393706675834]], dtype=object)

In [18]:
p1 - rot_disented @ p2

array([[0],
       [0],
       [1.77635683940025e-15]], dtype=object)

p2をp1に移す回転ベクトルに収束した！

In [19]:
print(R.from_matrix(rot1).as_rotvec())
print(omega[0])

[0.48647923 0.38485157 0.07752532]
[0.4303644  0.3436757  0.00758861]


ただし、回転ベクトル自体は$L$の計算に使ったものとは違う値になってる

2点でコストを定義すると、元の回転ベクトルに収束するとのことで試す↓

In [21]:
p4 = p.subs(x, 7).subs(y, 8).subs(z, 9)
p4_wedge = sympy.matrix2numpy(A.subs(k, p4[0]).subs(l, p4[1]).subs(m, p4[2]))

In [22]:
p3 = sympy.matrix2numpy(rot1 @ p4)

In [23]:
alpha = 0.005
omega = np.ones(3)

print("初期回転ベクトル:{}".format(omega))
# 勾配降下法
for i in trange(6000):
    rot = R.from_rotvec(omega)
    d_theta = -alpha * (p1.T @ rot.as_matrix().reshape(3,3) @ p2_wedge + p3.T @ rot.as_matrix().reshape(3,3) @ p4_wedge)
    tmp = R.from_rotvec(d_theta)
    tmp = tmp.as_matrix() @ rot.as_matrix()
    tmp = R.from_matrix(tmp)
    omega = tmp.as_rotvec()

初期回転ベクトル:[1. 1. 1.]


  0%|          | 0/6000 [00:00<?, ?it/s]

In [24]:
print(R.from_matrix(rot1).as_rotvec())
print(omega[0])

[0.48647923 0.38485157 0.07752532]
[0.48565578 0.38426023 0.0765845 ]


おんなじ（くらいの）値になった！！

In [27]:
# 実はこれでもいけるし計算速い
omega = np.ones(3)
rot = R.from_rotvec(omega)
for i in trange(6000):
    d_theta = -alpha * (p1.T @ rot.as_matrix().reshape(3,3) @ p2_wedge + p3.T @ rot.as_matrix().reshape(3,3) @ p4_wedge)
    rot = R.from_rotvec(d_theta) * rot
omega = rot.as_rotvec()

  0%|          | 0/6000 [00:00<?, ?it/s]

In [28]:
omega

array([[0.48565578, 0.38426023, 0.0765845 ]])